# Open AI Functions

Playing with Open AI Functions: https://openai.com/blog/function-calling-and-other-api-updates

This update was released on `June 13, 2023`.

To make the function calling easier, I am using [`ToolGPT`](https://github.com/tehruhn/ToolGPT).

### Import Dependencies

In [1]:
import os
import json
import openai
import pandas as pd

from dotenv import load_dotenv
from entsoe import EntsoePandasClient
from ToolGPT import ChatGPTWithFunctions

from functions.crypto import get_crypto_peak_price
from functions.entsoe_power import get_all_market_codes, market_area_to_market_code, calculate_mean_price
from functions.my_machine import my_machine_os
from functions.property_dataset import get_property_location_area_dataset

### Load and Set API Keys

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = EntsoePandasClient(api_key=os.getenv("ENTSOE_API_KEY"))

# Prompting Open AI's Model

Function calling is released for the following OpenAI models:

- `gpt-4-0613`
- `gpt-4-32k-0613`
- `gpt-3.5-turbo-0613`

In our case, we are going to test the new functionality on `gpt-3.5-turbo-0613`.

# GPT-3.5 vs. GPT-3.5 with Functions

## New/Recent Data

The OpenAI models are trained on historical data up to September 2021, so how does it work when trying to ask for new or recent information?

Question: `What was the peak price of Bitcoin yesterday?`

In [3]:
prompt = "What was the peak price of Bitcoin yesterday?"

chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}])
print(chat_completion.choices[0].message.content)

I'm sorry, but I cannot provide real-time financial data as I am an AI language model and do not have access to real-time information. Please consult a reliable financial news source or cryptocurrency exchange for current Bitcoin prices and information.


In [4]:
prompt = "What was the peak price of Bitcoin yesterday?"

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    get_crypto_peak_price])
print(ans)

What was the peak price of Bitcoin yesterday?
Received FUNCTION_CALL signal
{
  "cryptocurrency": "Bitcoin"
}
The peak price of Bitcoin yesterday was $26,676.18.
Received STOP signal
None


## Device Specific Information

You want to install new tool or software on your machine, but there exists multiple guides depending on your operating system or hardware profile.

Question: `How do I install docker onto my physical machine?`


In [5]:
prompt = "How do I install docker onto my physical machine?"

chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}])
print(chat_completion.choices[0].message.content)

As an AI language model, I cannot install software on your physical machine for you, but I can provide you with a general guide on how to install Docker on your physical machine.

Follow these general steps:

1. Check the minimum requirements for Docker on your operating system.
2. Download the appropriate Docker installer for your operating system from the Docker website.
3. Run the Docker installer, follow the on-screen instructions to complete the installation process.
4. Start the Docker engine and check that it's properly installed and running by running the "docker run hello-world" command.
5. If the hello-world image downloads and executes successfully, Docker is ready to use.

Note: For more detailed instructions, refer to the Docker documentation that corresponds to your OS.


In [6]:
prompt = "How do I install docker onto my physical machine?"

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    my_machine_os])
print(ans)

How do I install docker onto my physical machine?
Received FUNCTION_CALL signal
{
  "not_used": ""
}
My system is: Linux
To install Docker on a Linux machine, you can follow these steps:

1. Update the package index:
   ```
   sudo apt update
   ```

2. Install the necessary packages to allow apt to use a repository over HTTPS:
   ```
   sudo apt install apt-transport-https ca-certificates curl software-properties-common
   ```

3. Add the Docker GPG key:
   ```
   curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
   ```

4. Add the Docker repository:
   ```
   echo "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
   ```

5. Update the package index again:
   ```
   sudo apt update
   ```

6. Install Docker:
   ```
   sudo apt install docker-ce docker-ce-cli c

# Prompting Functions with Direct Answers

In the next example, we're going to prompt questions and pass functions with simple answers to retrieve the results.

Questions:
1. `What is the average spot price of Trondheim from January 1st 2023 to January 15th 2023?`
2. `Comparing the Trondheim and Oslo market areas, which one had the lowest average spot price from 2019 to 2022, and what were their respective average values?`
3. `Comparing all the market areas, which one had the lowest average spot price from 2019 to 2022, and range them from cheapest to highest with their respective values?`

In [7]:
prompt = "What is the average spot price of Trondheim from January 1st 2023 to January 15th 2023?"

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    market_area_to_market_code,
    calculate_mean_price])
print(ans)

What is the average spot price of Trondheim from January 1st 2023 to January 15th 2023?
Received FUNCTION_CALL signal
{
  "market_area": "Trondheim"
}
Received FUNCTION_CALL signal
{
  "market_code": "NO_3",
  "start_date": "2023-01-01",
  "end_date": "2023-01-15"
}
2023-01-01 00:00:00+01:00    23.35
2023-01-01 01:00:00+01:00    23.28
2023-01-01 02:00:00+01:00    23.37
2023-01-01 03:00:00+01:00    23.67
2023-01-01 04:00:00+01:00    24.12
                             ...  
2023-01-14 20:00:00+01:00    59.47
2023-01-14 21:00:00+01:00    51.98
2023-01-14 22:00:00+01:00    42.12
2023-01-14 23:00:00+01:00    35.32
2023-01-15 00:00:00+01:00    27.40
Freq: 60T, Length: 337, dtype: float64
74.80759643916915
<class 'pandas.core.series.Series'>
The average spot price of Trondheim from January 1st 2023 to January 15th 2023 is 74.81.
Received STOP signal
None


In [8]:
prompt = "Comparing the Trondheim and Oslo market areas, which one had the lowest average spot price from 2019 to 2022, and what were their respective average values?"

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    market_area_to_market_code,
    calculate_mean_price])
print(ans)

Comparing the Trondheim and Oslo market areas, which one had the lowest average spot price from 2019 to 2022, and what were their respective average values?
Received FUNCTION_CALL signal
{
  "market_area": "Trondheim"
}
Received FUNCTION_CALL signal
{
  "market_area": "Oslo"
}
Received FUNCTION_CALL signal
{
  "market_code": "NO_3",
  "start_date": "2019-01-01",
  "end_date": "2022-12-31"
}
2019-01-01 00:00:00+01:00    45.36
2019-01-01 01:00:00+01:00    45.31
2019-01-01 02:00:00+01:00    45.37
2019-01-01 03:00:00+01:00    45.21
2019-01-01 04:00:00+01:00    44.90
                             ...  
2022-12-30 20:00:00+01:00    24.01
2022-12-30 21:00:00+01:00    23.49
2022-12-30 22:00:00+01:00    23.39
2022-12-30 23:00:00+01:00    23.21
2022-12-31 00:00:00+01:00    19.94
Length: 35041, dtype: float64
32.74212836391655
<class 'pandas.core.series.Series'>
Received FUNCTION_CALL signal
{
  "market_code": "NO_1",
  "start_date": "2019-01-01",
  "end_date": "2022-12-31"
}
2019-01-01 00:00:00+0

In [10]:
prompt = "Comparing all the market areas, which one had the lowest average spot price from 2019 to 2022, and range them from cheapest to highest with their respective values?"

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    get_all_market_codes,
    market_area_to_market_code,
    calculate_mean_price])
print(ans)

Comparing all the market areas, which one had the lowest average spot price from 2019 to 2022, and range them from cheapest to highest with their respective values?
Received FUNCTION_CALL signal
{
  "not_used": ""
}
Received FUNCTION_CALL signal
{
  "market_code": "NO_1",
  "start_date": "2019-01-01",
  "end_date": "2022-12-31"
}
2019-01-01 00:00:00+01:00     48.77
2019-01-01 01:00:00+01:00     49.25
2019-01-01 02:00:00+01:00     49.17
2019-01-01 03:00:00+01:00     48.37
2019-01-01 04:00:00+01:00     47.19
                              ...  
2022-12-30 20:00:00+01:00    104.94
2022-12-30 21:00:00+01:00    106.10
2022-12-30 22:00:00+01:00     80.85
2022-12-30 23:00:00+01:00      1.77
2022-12-31 00:00:00+01:00     58.90
Length: 35041, dtype: float64
78.87352501355556
<class 'pandas.core.series.Series'>
Received FUNCTION_CALL signal
{
  "market_code": "NO_2",
  "start_date": "2019-01-01",
  "end_date": "2022-12-31"
}
2019-01-01 00:00:00+01:00     48.77
2019-01-01 01:00:00+01:00     49.25


## Prompting Functions with Indirect Answer

What about a situation where we do not have direct answers in the functions. Instead we e.g. have a DataFrame that is returned to the user, does the OpenAI model understand how to manipulate the data to obtain the wanted information?

Questions:
1. `Obtain a dataset from file containing a pandas dataframe with property listing information such as date, location, type, total price, number of bedrooms, size, and ad link. Read the file found in 'data/results.csv', and find out how many properties are listed in Oslo.`
2. `Read the file found in 'data/results.csv', and find out which property in Trondheim has the most bedrooms.`

In [11]:
prompt = "Obtain a dataset from file containing a pandas dataframe with property listing information such as date, location, type, total price, number of bedrooms, size, and ad link. Read the file found in 'data/results.csv', and find out how many properties are listed in Oslo."

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    get_property_location_area_dataset])
print(ans)

Obtain a dataset from file containing a pandas dataframe with property listing information such as date, location, type, total price, number of bedrooms, size, and ad link. Read the file found in 'data/results.csv', and find out how many properties are listed in Oslo.
Received FUNCTION_CALL signal
{
  "filepath": "data/results.csv",
  "location": "Oslo"
}


/home/fredrik/Documents/github/openai_functions/functions/property_dataset.py:6: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


There are 19,740 properties listed in Oslo.
Received STOP signal
None


In [12]:
prompt = "Read the file found in 'data/results.csv', and find out which property in Trondheim has the most bedrooms."

wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [
    get_property_location_area_dataset])
print(ans)

Read the file found in 'data/results.csv', and find out which property in Trondheim has the most bedrooms.
Received FUNCTION_CALL signal
{
  "filepath": "data/results.csv",
  "location": "Trondheim"
}


/home/fredrik/Documents/github/openai_functions/functions/property_dataset.py:6: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


The property in Trondheim with the most bedrooms is a house in Finnås with 5 bedrooms.
Received STOP signal
None


### Looks good, but...

Let's do some sanity checks!

In [13]:
df = get_property_location_area_dataset('data/results.csv', '')

df_oslo = df[df['location'] == 'Oslo']
print(f'Oslo: {df_oslo.shape}')

df_trd = df[df['location'] == 'Trondheim']
df_trd = df_trd.loc[df_trd['number_of_bedrooms'].idxmax()]
print(f'Trondheim:\n{df_trd}')

Oslo: (3861, 7)
Trondheim:
date                                                                2023-05-05
location                                                             Trondheim
property_type_description                                             Enebolig
price_total                     {'amount': 14863842.0, 'currency_code': 'NOK'}
number_of_bedrooms                                                        16.0
area_range                   {'description': 'size', 'size_from': 487.0, 's...
ad_link                      https://www.finn.no/realestate/homes/ad.html?f...
Name: 2405, dtype: object


/home/fredrik/Documents/github/openai_functions/functions/property_dataset.py:6: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
